# Notebook for the compuation of the hash-based similarities

Containing methods and functionality for computing and measuring similarities from the hashes

In [16]:
from experiments.grid_resolution import plot_grid_res_layers
from experiments.disk_resolution import plot_disk_dia_layers
from experiments.disk_resolution import plot_disk_numbers

import global_variables

In [17]:
#Test-plot
plot_grid_res_layers("porto", [1,2,3,4], [0.2, 5, 0.2], parallell_jobs=20, reference="frechet")


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 441 and the array at index 1 has size 2500

## Grid Porto

In [ ]:
# Porto/DTW - Frechet
plot_grid_res_layers("porto", [1,2,3,4,5,6,8,10], [0.2, 5, 0.2], parallell_jobs=20, reference="frechet")

In [ ]:
# Porto/DTW - DTW
plot_grid_res_layers("porto", [1,2,3,4,5,6,8,10], [0.2, 5, 0.2], parallell_jobs=20, reference="dtw")

In [ ]:
# Porto/ED - Frechet
plot_grid_res_layers("porto", [1,2,3,4,5,6,8,10], [0.2, 5, 0.2], measure="py_ed", parallell_jobs=20, reference="frechet")

In [ ]:
# Porto/ED - DTW
plot_grid_res_layers("porto", [1,2,3,4,5,6,8,10], [0.2, 5, 0.2], measure="py_ed", parallell_jobs=20, reference="dtw")

# Disk Porto

In [ ]:
plot_disk_dia_layers("porto", [1,2,3,4,5,6], [1, 3, 0.2], measure="py_dtw", reference="dtw", parallell_jobs=20 )

In [ ]:
plot_disk_dia_layers("porto", [1,2,3,4,5,6], [1, 3, 0.2], measure="py_dtw", reference="frechet", parallell_jobs=20 )

In [ ]:
plot_disk_dia_layers("porto", [1,2,3,4,5,6], [1, 3, 0.2], measure="py_ed", reference="dtw", parallell_jobs=20 )

In [ ]:
plot_disk_dia_layers("porto", [1,2,3,4,5,6], [1, 3, 0.2], measure="py_ed", reference="frechet", parallell_jobs=20 )

# Plotting disk number figure

In [ ]:
plot_disk_numbers("porto", 4, 2.2, [10,20,30,40,50,60,70,80,90,100], parallell_jobs=30)

# Computing the hash similarities that will be used for further research

In [18]:
from experiments.grid_similarity import generate_grid_hash_similarity
from experiments.disk_similarity import generate_disk_hash_similarity
import os

In [19]:
# Grid Porto
similarities = generate_grid_hash_similarity("porto", 1.6, 5)
output_path = "../code/experiments/similarities/grid_porto.csv"
similarities.to_csv(os.path.abspath(output_path))

In [4]:
# Disk Porto
similarities = generate_disk_hash_similarity("porto", 2.2, 4, 60)
output_path = "../code/experiments/similarities/disk_porto.csv"
similarities.to_csv(os.path.abspath(output_path))

Exception: Cant find file ../data/hashed_data/grid/porto/META-1000.txt

# Runtimes of grid similarity computations

In [20]:
from experiments.grid_similarity import measure_grid_hash_similarity_computation_time
from utils.figure_creator import draw_hash_similarity_runtime, draw_hash_similarity_runtime_logarithmic
import pandas as pd
import os
import numpy as np

In [24]:
# Measuring Grid Porto similarity computation times // idun (1node, 6cores, 128GB): 9min

runs = 10
data_sets = range(10,110,10)
output_folder = "../code/experiments/timing/"
file_name = "similarity_runtimes_grid_porto.csv"

df = pd.DataFrame( index=[f"run_{x+1}" for x in range(runs)], columns=[x for x in data_sets])

for size in data_sets:
    print(f"Computing size {size}", end="\r")
    execution_times = measure_grid_hash_similarity_computation_time("porto", size, 1.6, 5, "dtw", parallell_jobs=10)
    df[size] = execution_times

df.to_csv(os.path.join(output_folder, file_name))
df

,10,20,30,40,50,60,70,80,90,100
run_1,1.279667,1.310186,1.387999,1.247891,1.361872,1.341656,1.352935,1.268300,1.323448,1.304557
run_2,1.284826,1.305681,1.386045,1.248853,1.359759,1.333218,1.353737,1.267998,1.322378,1.307948
run_3,1.282063,1.310134,1.393171,1.248483,1.360821,1.350921,1.352306,1.266009,1.316501,1.306265
run_4,1.287920,1.307669,1.401757,1.250610,1.361570,1.333102,1.340792,1.268155,1.319134,1.302410
run_5,1.281075,1.307759,1.404818,1.247599,1.360523,1.329841,1.368645,1.268575,1.310928,1.306390
run_6,1.284560,1.308934,1.404397,1.250444,1.357526,1.339741,1.348219,1.263679,1.312871,1.304670
run_7,1.280419,1.307436,1.404487,1.246958,1.360346,1.344135,1.342103,1.265313,1.308402,1.306676
run_8,1.281778,1.306288,1.396653,1.244236,1.362987,1.333591,1.350049,1.265527,1.311252,1.289270
run_9,1.278014,1.305796,1.423040,1.251824,1.351012,1.331233,1.343245,1.267346,1.307228,1.295740
run_10,1.282495,1.315276,1.404965,1.248096,1.356245,1.332620,1.351751,1.260989,1.316807,1.289380


In [25]:
#SAME AS OVER, BUT FOR 1 RUN

# Measuring Grid Porto similarity computation times // idun (1node, 6cores, 128GB): 9min

output_folder = "../code/experiments/timing/"
file_name = f"similarity_runtimes_grid_porto-{global_variables.CHOSEN_SUBSET_NAME}.csv"

df = pd.DataFrame( index=["run_1"], columns=[global_variables.CHOSEN_SUBSET_NAME])


print(f"Computing {global_variables.CHOSEN_SUBSET_NAME}", end="\r")
execution_times = measure_grid_hash_similarity_computation_time("porto", global_variables.CHOSEN_SUBSET_SIZE, 1.6, 5, "dtw", parallell_jobs=1)
df[global_variables.CHOSEN_SUBSET_SIZE] = execution_times

df.to_csv(os.path.join(output_folder, file_name))
df

,subset-100,100
run_1,NaN,1.273122


In [26]:
# Generating a figure visualising the computation runtime of the hashes over Porto
from utils.figure_creator import draw_hash_similarity_runtime_logarithmic
import os
grid_path = os.path.abspath("../code/experiments/timing/similarity_runtimes_grid_porto.csv")
disk_path = os.path.abspath("../code/experiments/timing/similarity_runtimes_disk_porto.csv")
referencepath = os.path.abspath("../code/experiments/timing/similarity_runtimes_true_dtw_porto.csv")
city = "Porto"
draw_hash_similarity_runtime_logarithmic(city, grid_path, disk_path, path_to_reference=referencepath)

FileNotFoundError: [Errno 2] No such file or directory: '/cluster/home/andrehva/master/code/experiments/timing/similarity_runtimes_disk_porto.csv'

# Runtimes of disk similarity computations

In [12]:
from experiments.disk_similarity import measure_disk_hash_similarity_computation_time
import pandas as pd

In [ ]:
# Measuring Disk Porto similarity computation times // amalie, lokalt: 105min // idun (1node, 6cores, 128GB): 59min

runs = 10
data_sets = range(100,1001,100)
output_folder = "../code/experiments/timing/"
file_name = "similarity_runtimes_disk_porto.csv"

df = pd.DataFrame(index=[f"run_{x+1}" for x in range(runs)], columns=[x for x in data_sets])

for size in data_sets:
    print(f"Computing size {size}", end="\r")
    execution_times = measure_disk_hash_similarity_computation_time("porto", size, 2.2, 4, 60,"kd", measure="dtw", parallell_jobs=10)
    df[size] = execution_times

df.to_csv(os.path.join(output_folder, file_name))
df

# Computing variance from 10 different runs

In [27]:
from experiments.correlation import compute_correlation_similarity

In [28]:
compute_correlation_similarity("porto", "grid", 10)

Run : 0
Run : 1
Run : 2
Run : 3
Run : 4
Run : 5
Run : 6
Run : 7
Run : 8
Run : 9
porto grid : (min, max, avg, std)
DTW: 0.5481780170044595 0.5934460853484671 0.5691611920109005 0.014952889640567856
FRE: 0.6961329741011646 0.7386663851778994 0.7137623080623746 0.014302696461248916


In [15]:
# idun(samme): 24min
compute_correlation_similarity("porto", "disk", 10)

Run : 0


Exception: Cant find file ../data/hashed_data/grid/porto/META-1000.txt